<a href="https://colab.research.google.com/github/fadel11-hub/Tim-Coba-Coba/blob/main/audio_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Audio To Text

##Whisper (Open AI)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install openai-whisper

In [ ]:
import whisper

# Load the Whisper model
# This might take some time on the first run as it downloads the model
model_whisper = whisper.load_model("base")

# Transcribe the audio file
# Make sure you have an audio file named "audio.mp3" in your environment
result = model_whisper.transcribe("/content/drive/MyDrive/Datathon/Tim/Dokumen/Dataset/common_voice_id_25248253.mp3")

# Print the transcribed text
print(result["text"])

100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 93.3MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 ada di lantai 3 bangunan ini


#Model Klasifikasi

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
#  Load CSV dataset
df = pd.read_csv("/content/drive/MyDrive/Datathon/Tim/Dokumen/Dataset/darurat_aman_dataset.csv")  # Pastikan file ini ada

print(df)


In [ ]:
texts = df['text'].astype(str).tolist()
# Map 'aman' to 0 and 'darurat' to 1
label_mapping = {'aman': 0, 'darurat': 1}
labels = df['label'].map(label_mapping).astype(int).tolist()

In [ ]:

tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(texts)


word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(texts)
data_padded = pad_sequences(sequences, padding='post', truncating='post')

print('\n Word index =', word_index)
print('\n Sequences =', sequences)
print('\n Padded Sequences:')
print(data_padded)

# # Convert the labels lists into numpy arrays
data_labels = np.array(labels)
print(data_labels)

##Spilt Train

In [ ]:
from sklearn.model_selection import train_test_split # <--- IMPOR PENTING


X_train, X_test, y_train, y_test = train_test_split(
    data_padded, data_labels, test_size=0.25, random_state=42
)

print("\n--- Hasil Pembagian Data ---")
print(f"Shape X_train: {X_train.shape}")
print(f"Shape y_train: {y_train.shape}")
print(f"Shape X_test (Validation): {X_test.shape}")
print(f"Shape y_test (Validation Labels): {y_test.shape}")

## Threshold

In [ ]:
# Number of examples to use for training
training_size = 2000

# Vocabulary size of the tokenizer
vocab_size = 1000

# Maximum length of the padded sequences
max_length = 20

# Output dimensions of the Embedding layer
embedding_dim = 16

# Training data
num_epochs = 30

## Multilayer Perceptron

### Tensorflow

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling1D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    GlobalAveragePooling1D(),  # rata-rata embedding
    Dropout(0.5),
    Dense(6, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])


In [ ]:
# Compile the model
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
num_epochs = 30

# Train the model
history = model.fit(X_train, y_train, epochs=num_epochs, verbose=2, validation_data=(X_test, y_test))

### Sckit Learn

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score


In [ ]:
# Misal kolom 'feature1' dan 'feature2' bertipe string
label_encoders = {}
for col in ['text', 'label']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le


In [ ]:
target_Encoder = LabelEncoder()
df['label'] = target_Encoder.fit_transform(df['label'])

In [ ]:
X = df.drop('label', axis=1)
y = df['label']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(100,),  # 1 hidden layer
                    max_iter=500,
                    activation='relu',
                    solver='adam',
                     alpha=0.0001,
                    random_state=42,)

mlp.fit(X_train_scaled, y_train)


In [ ]:
y_pred = mlp.predict(X_test_scaled)

print("Akurasi:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


In [ ]:
import joblib

# Save scaler & model
# --- Load model dan preprocessing ---
mlp = joblib.load(mlp,"mlp_model.pkl")
scaler = joblib.load("scaler.pkl")
label_encoder = joblib.load("label_encoder.pkl")


## LSTM

In [ ]:
model_lstm = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    LSTM(64, return_sequences=False),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])
model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model_lstm.summary()

In [ ]:
model_lstm.fit(X_train, y_train, epochs=num_epochs, verbose=2, validation_data=(X_test, y_test))

In [ ]:
model_lstm.save("model_lstm.h5")

In [ ]:
import pickle
with open('tokenizer.pickle', 'wb') as f:
    pickle.dump(tokenizer, f)
with open('label_encoder.pickle', 'wb') as f:
    pickle.dump(label_encoder, f)

## Test Model

In [ ]:
# Transcribe the audio file
# Make sure you have an audio file named "audio.mp3" in your environment
result2 = model_whisper.transcribe("/content/drive/MyDrive/Datathon/Tim/Dokumen/Dataset/Darurat_baru.m4a")

# Print the transcribed text
print(result2["text"])

In [ ]:
# --- 5. Prediksi Teks dari Whisper ---
input_seq = tokenizer.texts_to_sequences(result2['text'])
input_pad = pad_sequences(input_seq, maxlen=10, padding='post')

pred = mlp.predict(input_pad)[0][0]
print("Status:", "🚨 darurat" if pred > 0.5 else "✅ aman")

In [ ]:
mlp_pred = mlp.predict(result2['text'])
print(mlp_pred)
print("Status:", "🚨 darurat" if pred == 1 else "✅ aman")

##MLP

In [ ]:
# --- IMPORT LIBRARY ---
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score
import joblib

# --- LOAD DATASET CSV ---
data = pd.read_csv("/content/drive/MyDrive/Datathon/Tim/Dokumen/Dataset/darurat_aman_dataset.csv")

# ✅ Pisahkan fitur dan label
X_text = data["text"]  # kolom teks
y = data["label"]     # kolom label

# --- ENCODE TEXT KE NUMERIC (PAKAI TOKENIZER) ---
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_text)

# Save tokenizer
joblib.dump(tokenizer, "tokenizer.pkl")

X_seq = tokenizer.texts_to_sequences(X_text)
X_pad = pad_sequences(X_seq, maxlen=20, padding='post')

# --- SCALE FITUR ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_pad)

# Save scaler
joblib.dump(scaler, "scaler.pkl")

# --- ENCODE LABEL ---
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Save label encoder
joblib.dump(label_encoder, "label_encoder.pkl")

# --- SPLIT DATA ---
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42
)

# --- TRAINING MLP ---
mlp = MLPClassifier(hidden_layer_sizes=(128,), activation='relu', max_iter=300)
mlp.fit(X_train, y_train)

# Save model
joblib.dump(mlp, "mlp_model.pkl")

# --- EVALUASI ---
y_pred = mlp.predict(X_test)
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))
print("Akurasi:", accuracy_score(y_test, y_pred))



              precision    recall  f1-score   support

        aman       0.90      0.95      0.92        19
     darurat       0.95      0.91      0.93        22

    accuracy                           0.93        41
   macro avg       0.93      0.93      0.93        41
weighted avg       0.93      0.93      0.93        41

Akurasi: 0.926829268292683


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
# --- LOAD MODEL DAN PREPROCESSING ---
mlp = joblib.load("mlp_model.pkl")
scaler = joblib.load("scaler.pkl")
label_encoder = joblib.load("label_encoder.pkl")
tokenizer = joblib.load("tokenizer.pkl")

# --- TRANSKRIPSI AUDIO (WHISPER) ---
import whisper
model_whisper = whisper.load_model("base")

result = model_whisper.transcribe("/content/drive/MyDrive/Datathon/Tim/Dokumen/Dataset/common_voice_id_25248253.mp3")
result2 = model_whisper.transcribe("/content/drive/MyDrive/Datathon/Tim/Dokumen/Dataset/Darurat_baru.m4a")
# print("Hasil Transkrip:", result["text"])

input_text = "saya menonton vidio"
print("Input text:", input_text)

# --- PREPROCESS TEKS ---
seq = tokenizer.texts_to_sequences(input_text)
pad = pad_sequences(seq, maxlen=20, padding='post')
scaled = scaler.transform(pad)

# --- PREDIKSI KELAS ---
pred_encoded = mlp.predict(scaled)[0]
pred_label = label_encoder.inverse_transform([pred_encoded])[0]

print("Prediksi Label (Encoded):", pred_encoded)
print("Status:", "🚨 darurat" if pred_label == "darurat" else "✅ aman")
